# TensorRT Detection Demo

This notebook mirrors the ONNX demo but runs a TensorRT engine.
- Single image inference
- Folder batch inference
- Optional JetCam CSI camera loop


In [ ]:
from pathlib import Path
import importlib
import cv2
import numpy as np
import matplotlib.pyplot as plt

import trt_detector
importlib.reload(trt_detector)
from trt_detector import TrtYoloDetector
from onnx_detector import load_class_names


In [ ]:
# Paths and settings
repo_root = Path.cwd()

engine_path = repo_root / 'yolo11_signs.engine'  # change to your engine path
classes_path = repo_root / 'classes.txt'

conf_thres = 0.25
iou_thres = 0.45
imgsz = 640  # e.g. 640 or (640, 640)
measure_time = True
device_id = 0

detector = TrtYoloDetector(
    engine_path,
    class_names=load_class_names(classes_path),
    conf_thres=conf_thres,
    iou_thres=iou_thres,
    input_size=imgsz,
    device_id=device_id,
)


In [ ]:
def show_bgr(image_bgr, title=None):
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    if title:
        plt.title(title)
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()


In [ ]:
image_path = repo_root / 'test_images' / 'collect1_frame_000861.jpg'
image = cv2.imread(str(image_path))
if image is None:
    raise FileNotFoundError(image_path)

detections = detector.predict(image, measure_time=measure_time)
if measure_time and detector.last_inference_time is not None:
    inf_ms = detector.last_inference_time * 1000.0
    fps = 1.0 / detector.last_inference_time if detector.last_inference_time > 0 else float('inf')
    print(f"Inference: {inf_ms:.2f} ms | FPS: {fps:.2f}")
vis = detector.visualize(image, detections)
show_bgr(vis, title=f'dets: {len(detections)}')


In [ ]:
image_folder = repo_root / 'test_images'
save_dir = repo_root / 'runs' / 'trt_detect_demo'
save_dir.mkdir(parents=True, exist_ok=True)

count = 0
for img_path in sorted(image_folder.iterdir()):
    if img_path.suffix.lower() not in {'.jpg', '.jpeg', '.png', '.bmp'}:
        continue
    image = cv2.imread(str(img_path))
    if image is None:
        continue
    detections = detector.predict(image, measure_time=measure_time)
    if measure_time and detector.last_inference_time is not None:
        inf_ms = detector.last_inference_time * 1000.0
        fps = 1.0 / detector.last_inference_time if detector.last_inference_time > 0 else float('inf')
        print(f"Inference: {inf_ms:.2f} ms | FPS: {fps:.2f}")
    vis = detector.visualize(image, detections)
    out_path = save_dir / img_path.name
    cv2.imwrite(str(out_path), vis)
    count += 1
    if count >= 100:
        show_bgr(vis, title=img_path.name)
        break

print(f'Saved to: {save_dir}')


In [ ]:
from IPython.display import display, clear_output
from jetcam.csi_camera import CSICamera
import gc

camera = None

def iter_camera_frames(num_frames=50, width=640, height=480, fps=30):
    global camera
    camera = CSICamera(width=width, height=height, fps=fps)
    for _ in range(num_frames):
        frame = camera.read()
        if frame is None:
            continue
        yield frame


def stop_camera():
    global camera
    if camera is None:
        return
    if hasattr(camera, "stop"):
        camera.stop()
    if hasattr(camera, "release"):
        camera.release()
    camera = None
    gc.collect()

for frame in iter_camera_frames(num_frames=20):
    detections = detector.predict(frame, measure_time=measure_time)
    if measure_time and detector.last_inference_time is not None:
        inf_ms = detector.last_inference_time * 1000.0
        fps = 1.0 / detector.last_inference_time if detector.last_inference_time > 0 else float('inf')
        print(f"Inference: {inf_ms:.2f} ms | FPS: {fps:.2f}")
    vis = detector.visualize(frame, detections)
    clear_output(wait=True)
    plt.figure(figsize=(6, 4))
    plt.imshow(vis[:, :, ::-1])
    plt.axis('off')
    display(plt.gcf())
    plt.close()


In [ ]:
# Stop camera resources
try:
    stop_camera()
except NameError:
    pass
